### Import stuff

Run this notebook using the 'meanderpy' conda environment.

In [527]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.signal import savgol_filter
from scipy.spatial import distance
from librosa.sequence import dtw
from tqdm import trange, tqdm
import networkx as nx
from shapely.geometry import Polygon, MultiPolygon, Point, MultiLineString, LineString, shape, JOIN_STYLE
from shapely.geometry.polygon import LinearRing
from shapely.ops import snap, unary_union

In [518]:
!which jupyter

/Users/zoltan/mambaforge/envs/meanderpy/bin/jupyter


In [3]:
cd /Users/zoltan/Dropbox/Channels/meanderpy/meanderpy

/Users/zoltan/Dropbox/Channels/meanderpy/meanderpy


In [5]:
import meanderpy as mp

In [6]:
cd /Users/zoltan/Dropbox/meandergraph

/Users/zoltan/Dropbox/Meandergraph


In [7]:
import meandergraph as mg

In [8]:
%matplotlib qt

In [9]:
import sys
print(sys.executable)

/Users/zoltan/mambaforge/envs/meanderpy/bin/python


In [10]:
import geopandas as gpd

In [11]:
from glob import glob #for finding files that match a certain string pattern
lbs = sorted(glob("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb*.shp"))
rbs = sorted(glob("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/rb*.shp"))

In [491]:
lbs

['/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1986.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1987.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1988.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1989.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1990.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1991.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1992.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1993.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1994.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1995.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1996.shp',
 '/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/lb_1997.shp',
 '/Users/zoltan/

In [56]:
X1 = []
X2 = []
Y1 = []
Y2 = []
for fname in lbs:
    lb = gpd.read_file(fname)
    X2.append(lb['geometry'][0].xy[0])
    Y2.append(lb['geometry'][0].xy[1])
for fname in rbs:
    rb = gpd.read_file(fname)
    X1.append(rb['geometry'][0].xy[0])
    Y1.append(rb['geometry'][0].xy[1])

In [57]:
# plot banklines
plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal');

In [58]:
points = plt.ginput(n=2) 
# click twice to select start- and endpoints on first bankline (does not matter which one)

In [59]:
points

[(281080.33683424315, -1669875.1764392515),
 (285412.3847311827, -1649967.2259915178)]

In [14]:
points = [(286850.39156848995, -1660631.8404876508),
 (284610.488612104, -1650178.960024516)]

In [60]:
from scipy import signal, spatial
cl_points = np.vstack((X1[0], Y1[0])).T # coordinates of first centerlines
tree = spatial.KDTree(cl_points)

plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal')
plt.plot(X1[0][tree.query(points[0])[1]], Y1[0][tree.query(points[0])[1]], 
         'ro', zorder=10000)
plt.plot(X1[0][tree.query(points[1])[1]], Y1[0][tree.query(points[1])[1]], 
         'ro', zorder=10000)

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [61]:
first_index = tree.query(points[0])[1]
last_index = tree.query(points[1])[1]

first_channel = 0
last_channel = 31

# correlate all lines:    
P1 = []
Q1 = []
for i in trange(len(X1) - 1):
    p, q = mg.correlate_curves(X1[i], X1[i+1], Y1[i], Y1[i+1])
    P1.append(p)
    Q1.append(q)

# correlate all lines:    
P2 = []
Q2 = []
for i in trange(len(X2) - 1):
    p, q = mg.correlate_curves(X2[i], X2[i+1], Y2[i], Y2[i+1])
    P2.append(p)
    Q2.append(q)
    
indices1, x, y = mg.find_indices(first_index, X1, Y1, P1, Q1)
indices2, x, y = mg.find_indices(last_index, X1, Y1, P1, Q1)
for i in range(len(X1)):
    X1[i] = X1[i][indices1[i] : indices2[i]+1]
    Y1[i] = Y1[i][indices1[i] : indices2[i]+1]

indices1, x, y = mg.find_indices(first_index, X2, Y2, P2, Q2)
indices2, x, y = mg.find_indices(last_index, X2, Y2, P2, Q2)
for i in range(len(X2)):
    X2[i] = X2[i][indices1[i] : indices2[i]+1]
    Y2[i] = Y2[i][indices1[i] : indices2[i]+1]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:49<00:00,  1.66s/it]


### Resample and correlate centerlines and banklines

In [62]:
# create Z coordinates - just a bunch of zeros -- needed for resampling function in meanderpy:
Z1 = []
for i in range(len(X1)):
    Z1.append(np.zeros(np.shape(X1[i])))
Z2 = []
for i in range(len(X2)):
    Z2.append(np.zeros(np.shape(X2[i])))

# resample centerlines to ds = 2.0 meters:
for i in range(len(X1)):
    x,y,z,dx,dy,dz,ds,s = mp.resample_centerline(X1[i], Y1[i], Z1[i], 2.0)
    X1[i] = x
    Y1[i] = y
for i in range(len(X2)):
    x,y,z,dx,dy,dz,ds,s = mp.resample_centerline(X2[i], Y2[i], Z2[i], 2.0)
    X2[i] = x
    Y2[i] = y

P1, Q1 = mg.correlate_set_of_curves(X1, Y1)
P2, Q2 = mg.correlate_set_of_curves(X2, Y2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [01:52<00:00,  3.76s/it]


In [63]:
plt.figure()
plt.plot(X1[10], Y1[10])
plt.plot(X2[10], Y2[10])
plt.axis('equal');

In [20]:
from importlib import reload
reload(mg)

<module 'meandergraph' from '/Users/zoltan/Dropbox/Meandergraph/meandergraph/meandergraph.py'>

### Create centerline- and bank graphs

In [65]:
# reload(mg)
ts = len(X1)
# graph = mg.create_graph_from_channel_lines(X[:ts], Y[:ts], P[:ts-1], Q[:ts-1], n_points=20, max_dist=100, remove_cutoff_edges=True)
graph1 = mg.create_graph_from_channel_lines(X1[:ts], Y1[:ts], P1[:ts-1], Q1[:ts-1], n_points=20, max_dist=1000, remove_cutoff_edges=True)
graph2 = mg.create_graph_from_channel_lines(X2[:ts], Y2[:ts], P2[:ts-1], Q2[:ts-1], n_points=20, max_dist=1000, remove_cutoff_edges=True)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 744.44it/s]


In [66]:
# graph = mg.remove_high_density_nodes(graph, min_dist = 10, max_dist = 30)
graph1 = mg.remove_high_density_nodes(graph1, min_dist = 20, max_dist = 60)
graph2 = mg.remove_high_density_nodes(graph2, min_dist = 20, max_dist = 60)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 117.18it/s]


### Plot one of the graphs

In [86]:
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_graph(graph2, ax)
plt.axis('equal');

In [382]:
# save graphs
nx.write_gpickle(graph1, "graph1.gpickle")
nx.write_gpickle(graph2, "graph2.gpickle")

In [293]:
reload(mg)
fig = plt.figure()
ax = fig.add_subplot(111)
# erosion_bars, chs, all_chs, jumps, cutoffs = mg.plot_erosion(graph1, graph2, cutoff_area, ax)
bars, chs, all_chs, jumps, cutoffs = mg.plot_bars3(graph1, graph2, cutoff_area, ax)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:01<00:00, 26.10it/s]


### Create and plot scrolls and bars (connected scrolls)

In [414]:
from importlib import reload
reload(mg)
cutoff_area = 1*1e6
scrolls, scroll_ages, cutoffs, all_bars_graph = mg.create_scrolls_and_find_connected_scrolls(graph1, graph2, cutoff_area)
plt.axis('equal');

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:02<00:00, 14.70it/s]


### Create and plot 'bar graphs', colored by migration rate

In [585]:
# create bars and bar graphs
reload(mg)
min_area = 1000
wbars, poly_graph_1, poly_graph_2 = mg.create_polygon_graphs_and_bar_graphs(graph1, graph2, all_bars_graph, 
                                                    scrolls, scroll_ages, cutoffs, X1, Y1, X2, Y2, min_area)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:13<00:00,  1.50it/s]


In [490]:
# plot them, using migration rate
fig = plt.figure(figsize = (12, 12)) 
ax = fig.add_subplot(111)
mg.plot_bar_graphs(graph1, graph2, wbars, ts, cutoffs, 356*24*60*60, X1, Y1, X2, Y2, 300, 1, 0, 500, 'migration', ax)
ax.set_adjustable("box")
ax.axis('equal')
fig.tight_layout()

# ax.plot(X1[ts-1], Y1[ts-1], 'r') # this is needed because Adobe Illustrator does not like the most recent channel polygon
# ax.plot(X2[ts-1], Y2[ts-1], 'r')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:03<00:00,  6.07it/s]


In [349]:
# plot one of the polygon graphs
fig = plt.figure()
ax = fig.add_subplot(111)
for node in poly_graph_1.nodes:
    poly = poly_graph_1.nodes[node]['poly']
    if type(poly) == MultiPolygon:
        for p in poly.geoms:
            ax.fill(p.exterior.xy[0], p.exterior.xy[1], alpha=0.5, color = 'lightblue')
    else:
        ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], alpha=0.5, color = 'lightblue')
plt.axis('equal');

In [353]:
import matplotlib as mpl
fig = plt.figure()
ax = fig.add_subplot(111)
norm = mpl.colors.Normalize(vmin=0, vmax=len(X1))
m = mpl.cm.ScalarMappable(norm=norm, cmap='viridis')
for node in poly_graph_1.nodes:
        poly = poly_graph_1.nodes[node]['poly']
        if type(poly) == Polygon:
            ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                facecolor = m.to_rgba(poly_graph_1.nodes[node]['age']), 
                edgecolor='k', linewidth=0.25, alpha=0.4)
plt.axis('equal')

(280424.35791427176,
 287652.17915516545,
 -1671170.212414464,
 -1649023.0974323864)

In [484]:
import matplotlib as mpl
fig = plt.figure()
ax = fig.add_subplot(111)
norm = mpl.colors.Normalize(vmin=0, vmax=len(X1))
m = mpl.cm.ScalarMappable(norm=norm, cmap='viridis')
for node in poly_graph_2.nodes:
        poly = poly_graph_2.nodes[node]['poly']
        if type(poly) == Polygon:
            ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                facecolor = m.to_rgba(poly_graph_2.nodes[node]['age']), 
                edgecolor='k', linewidth=0.25, alpha=0.4)
plt.axis('equal')

(280619.02578016854,
 287876.49629426346,
 -1671434.5072076914,
 -1648835.953634875)

2023-06-27 16:34:34.835 python[60378:3041780] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_bundleIdentifierWithReply:) block performed very slowly (3.04 secs).


In [495]:
fig = plt.figure()
ax = fig.add_subplot(111)
norm = mpl.colors.Normalize(vmin=-100, vmax=100)
m = mpl.cm.ScalarMappable(norm=norm, cmap='coolwarm_r')
for node in tqdm(poly_graph_2.nodes):
        poly = poly_graph_2.nodes[node]['poly']
        if type(poly) == Polygon:
            ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                facecolor = m.to_rgba(poly_graph_2.nodes[node]['migr_rate']*poly_graph_2.nodes[node]['direction']), 
                edgecolor='k', linewidth=0.25, alpha=0.4)
plt.axis('equal')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 28123/28123 [00:14<00:00, 1939.99it/s]


(280619.02578016854,
 287876.49629426346,
 -1671434.5072076914,
 -1648835.953634875)

In [496]:
fig = plt.figure()
ax = fig.add_subplot(111)
norm = mpl.colors.Normalize(vmin=-100, vmax=100)
m = mpl.cm.ScalarMappable(norm=norm, cmap='coolwarm')
for node in tqdm(poly_graph_1.nodes):
        poly = poly_graph_1.nodes[node]['poly']
        if type(poly) == Polygon:
            ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                facecolor = m.to_rgba(poly_graph_1.nodes[node]['migr_rate']*poly_graph_1.nodes[node]['direction']), 
                edgecolor='k', linewidth=0.25, alpha=0.4)
plt.axis('equal')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27542/27542 [00:06<00:00, 4270.79it/s]


(280424.35791427176,
 287652.17915516545,
 -1671170.212414464,
 -1649023.0974323864)

In [493]:
poly_graph_2.nodes[1000]

{'poly': <POLYGON ((282227.771 -1669664.025, 282264.217 -1669653.301, 282270.465 -166...>,
 'age': 8,
 'x': 282227.7706218711,
 'y': -1669664.0252079372,
 'length': 24.6031532153684,
 'width': 37.98129374466386,
 'direction': -1,
 'migr_rate': 24.6031532153684}

In [72]:
# plot the other polygon graph
fig = plt.figure()
ax = fig.add_subplot(111)
for node in poly_graph_2.nodes:
    poly = poly_graph_2.nodes[node]['poly']
    if type(poly) == MultiPolygon:
        for p in poly.geoms:
            ax.fill(p.exterior.xy[0], p.exterior.xy[1], alpha=0.5, color = 'lightblue')
    else:
        ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], alpha=0.5, color = 'lightblue')
plt.axis('equal');

In [102]:
def add_edge_directions_to_bank_graph(graph):
    for node in trange(graph.graph['number_of_centerlines']):
        path = mg.find_longitudinal_path(graph, node)
        for i in range(len(path) - 1):
            node_1 = path[i]
            node_2 = path[i+1]
            node_1_children = list(graph.successors(node_1))
            node_2_children = list(graph.successors(node_2))
            node_3 = False
            node_4 = False
            for n in node_1_children:
                if graph[node_1][n]['edge_type'] == 'radial':
                    node_4 = n
            for n in node_2_children:
                if graph[node_2][n]['edge_type'] == 'radial':
                    node_3 = n
            x1 = graph.nodes[node_1]['x']
            y1 = graph.nodes[node_1]['y']
            x2 = graph.nodes[node_2]['x']
            y2 = graph.nodes[node_2]['y']
            node_1_coords = np.array([x1, y1])
            node_2_coords = np.array([x2, y2])
            if node_3:
                x3 = graph.nodes[node_3]['x']
                y3 = graph.nodes[node_3]['y']
                node_3_coords = np.array([x3, y3])
                dist_23 = np.linalg.norm(node_2_coords - node_3_coords)
                direction_23 = mg.directionOfPoint(x1, y1, x2, y2, x3, y3)
                graph[node_2][node_3]['direction'] = direction_23
            if node_4:
                x4 = graph.nodes[node_4]['x']
                y4 = graph.nodes[node_4]['y']
                node_4_coords = np.array([x4, y4])
                dist_14 = np.linalg.norm(node_1_coords - node_4_coords)
                direction_14 = mg.directionOfPoint(x1, y1, x2, y2, x4, y4)
                graph[node_1][node_4]['direction'] = direction_14
    return graph


31

In [392]:
graph1 = add_edge_directions_to_bank_graph(graph1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 95.22it/s]


In [217]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.get_cmap("tab10")
for node in tqdm(graph1.graph['start_nodes']):
    path, path_ages = mg.find_radial_path(graph1, node)
    for i in range(len(path)-1):
        if graph1[path[i]][path[i+1]]['direction'] == 1:   # erosion on the right bank
            ax.plot(graph1.graph['x'][[path[i], path[i+1]]], 
                    graph1.graph['y'][[path[i], path[i+1]]], color=cmap(1))
        if graph1[path[i]][path[i+1]]['direction'] == -1:  # deposition on the right bank
            ax.plot(graph1.graph['x'][[path[i], path[i+1]]], 
                    graph1.graph['y'][[path[i], path[i+1]]], color=cmap(0))
plt.axis('equal');

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 1268/1268 [00:06<00:00, 206.14it/s]


In [220]:
plt.plot(graph1.graph['x'][path], graph1.graph['y'][path], 'k')

In [190]:
graph1.graph.keys()

dict_keys(['number_of_centerlines', 'x', 'y', 'cutoff_nodes', 'start_nodes'])

In [252]:
from matplotlib.colors import LinearSegmentedColormap
cmap2 = LinearSegmentedColormap.from_list('two_colors', [cmap(0), cmap(1)], N=2)

In [481]:
plt.figure()
# for i in range(0, 5): #graph1.graph['number_of_centerlines']):
migr_rates = []
dirs = []
curvs = []
path = mg.find_longitudinal_path(graph1, 19)
for node in path:
    path1, dummy = mg.find_radial_path(graph1, node)
    if len(path1) > 1:
        x1 = graph1.nodes[path1[0]]['x']
        y1 = graph1.nodes[path1[0]]['y']
        x2 = graph1.nodes[path1[1]]['x']
        y2 = graph1.nodes[path1[1]]['y']
        dist = ((x2-x1)**2 + (y2-y1)**2)**0.5
        direction = graph1[path1[0]][path1[1]]['direction']
        migr_rates.append(dist*direction)
        dirs.append(direction)
        curvs.append(graph1.nodes[node]['curv'])
plt.plot(migr_rates)
plt.plot(-1*np.array(curvs)*50000)
plt.plot([0, len(migr_rates)], [0, 0], 'r--')
# plt.figure()
# plt.scatter(np.arange(len(migr_rates)), np.cumsum(migr_rates), c=np.array(dirs), cmap=cmap2);

In [482]:
plt.figure()
plt.scatter(-1*np.array(curvs)*50000, migr_rates)

In [188]:
plt.figure()
# for node in long_path_nodes[300:350]:
path, path_ages = mg.find_radial_path(graph1, 3255)
dirs = []
lengths = []
for i in range(len(path)-1):
    dirs.append(graph1[path[i]][path[i+1]]['direction'])
    x1 = graph1.nodes[path[i]]['x']
    y1 = graph1.nodes[path[i]]['y']
    x2 = graph1.nodes[path[i+1]]['x']
    y2 = graph1.nodes[path[i+1]]['y']
    lengths.append(((x2-x1)**2 + (y2-y1)**2)**0.5)
plt.plot(np.array(dirs)*np.array(lengths))

In [185]:
long_path_nodes[100:110]

[3131, 3162, 3193, 3224, 3255, 3286, 3317, 3348, 3379, 3410]

In [123]:
# add curvature attribute to bankline graphs (needed for curvature maps)
mg.add_curvature_to_line_graph(graph1, smoothing_factor = 51)
mg.add_curvature_to_line_graph(graph2, smoothing_factor = 51)

In [456]:
graph1.nodes[1000]['curv']

0.0006098016037350975

### Plot migration rate map for one bar

In [41]:
reload(mg)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.fill(wbars[1].polygon.exterior.xy[0], wbars[1].polygon.exterior.xy[1])
mg.plot_migration_rate_map(wbars[1], graph1, graph2, vmin=0, vmax=300, dt=365*24*60*60, saved_ts=1, ax=ax)
plt.axis('equal');

### Plot age map for one bar

In [46]:
reload(mg)
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_age_map(wbars[1], vmin=0, vmax=ts, W=300, ax=ax)
plt.axis('equal');

### Plot bar polygons and their numbers

In [438]:
# plot bar polygons and their numbers
# fig = plt.figure(figsize = (15, 9))
# ax = fig.add_subplot(111)
# plt.figure(35)
for wbar in wbars:
#     wbar.add_bank_type()
    if wbar.bank_type == 'right':
        plt.fill(wbar.polygon.exterior.xy[0], wbar.polygon.exterior.xy[1], color='white')
count = 0
# for wbar in wbars:
#     ax.text(wbar.polygon.centroid.x, wbar.polygon.centroid.y, str(count), fontsize = 16)
#     count += 1
# plt.axis('equal');

In [446]:
type(right_bars)

shapely.geometry.multipolygon.MultiPolygon

In [453]:
# for wbar in wbars:
#     wbar.add_bank_type()

right_bars = []
for wbar in wbars:
    if wbar.bank_type == 'right':
        right_bars.append(wbar.polygon)
        
norm = mpl.colors.Normalize(vmin=0, vmax=len(X1))
m = mpl.cm.ScalarMappable(norm=norm, cmap='viridis')
        
fig = plt.figure()
ax = fig.add_subplot(111)
for node in tqdm(poly_graph_1.nodes):
        poly = poly_graph_1.nodes[node]['poly']
        if type(poly) == Polygon:
            plotting = True
            for bar in right_bars:
                if bar.buffer.intersects(poly):
                    plotting = False
            if plotting:
                if poly_graph_1.nodes[node]['direction'] == 1:
                    ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                       facecolor = m.to_rgba(poly_graph_1.nodes[node]['age']), 
                        edgecolor='k', linewidth=0.25, alpha=0.4)
#                 else:
#                     ax.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
#                         facecolor = cmap(0), 
#                         edgecolor='k', linewidth=0.25, alpha=0.4)
plt.axis('equal')
    
# for wbar in wbars:
#     if wbar.bank_type == 'right':
#         plt.fill(wbar.polygon.exterior.xy[0], wbar.polygon.exterior.xy[1], color='white')

ch = mg.create_channel_polygon_2(X1[-1], Y1[-1], X2[-1], Y2[-1])
plt.fill(ch.exterior.xy[0], ch.exterior.xy[1], 'b', alpha=0.2)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27542/27542 [00:19<00:00, 1416.44it/s]


In [526]:
right_bars

[<POLYGON ((281384.019 -1669514.51, 281356.238 -1669500.286, 281354.755 -1669...>,
 <POLYGON ((284715.269 -1665975.413, 284731.648 -1665974.773, 284742.405 -166...>,
 <POLYGON ((281757.304 -1660210.573, 281773.176 -1660240.785, 281784.8 -16602...>,
 <POLYGON ((284313.091 -1660408.327, 284323.383 -1660422.536, 284329.44 -1660...>,
 <POLYGON ((285954.893 -1656441.525, 285902.685 -1656481.144, 285864.661 -165...>,
 <POLYGON ((283162.52 -1652649.976, 283174.726 -1652654.181, 283210.048 -1652...>,
 <POLYGON ((280904.181 -1652985.183, 280939.277 -1653037.344, 280966.785 -165...>,
 <POLYGON ((284820.244 -1650277.936, 284814.095 -1650279.163, 284829.613 -165...>,
 <POLYGON ((284440.766 -1665944.559, 284454.839 -1665944.001, 284500.862 -166...>]

### Some debugging code

In [49]:
# code for debugging bar polygons
from shapely.geometry import Point

def debug_bar_polygons(wbar, graph1, graph2, dt, saved_ts, ax):
    mg.plot_migration_rate_map(wbar, graph1, graph2, vmin=0, vmax=40, dt=dt, saved_ts=saved_ts, ax=ax)
    plt.axis('equal');
    nodes = []
    if wbar.scrolls[-1].bank == 'right':
        line_graph = graph1
    else:
        line_graph = graph2
    for node in line_graph.nodes:
        point = Point(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'])
        if wbar.polygon.contains(point):
            nodes.append(node)
    for node in nodes:
        plt.plot(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'], 'k.')
        plt.text(line_graph.nodes[node]['x'], line_graph.nodes[node]['y'], str(node))

def debug_bar_polygons_2(wbar, graph1, graph2, dt, saved_ts, ax):
    mg.plot_migration_rate_map(wbar, graph1, graph2, vmin=0, vmax=300, dt=dt, saved_ts=saved_ts, ax=ax)
    plt.axis('equal');
    for node in wbar.bar_graph.nodes:
        ax.text(wbar.bar_graph.nodes[node]['poly'].centroid.x, wbar.bar_graph.nodes[node]['poly'].centroid.y, str(node))

In [50]:
dt=365*24*60*60
saved_ts = 1
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_graph(graph2, ax)
fig = debug_bar_polygons_2(wbars[10], graph1, graph2, dt, saved_ts, ax)
# debug_bar_polygons(wbars[10], graph1, graph2, dt, saved_ts, ax)


### Plot radial lines only

In [51]:
fig = plt.figure()
ax = fig.add_subplot(111)
path = mg.find_longitudinal_path(graph2, graph2.graph['start_nodes'][0])
for node in path:
    radial_path, dummy = mg.find_radial_path(graph2, node)
    x = graph2.graph['x'][radial_path]
    y = graph2.graph['y'][radial_path]
    plt.plot(x, y, 'k', linewidth = 0.5)
plt.axis('equal');

### Create and plot 'simple' polygon graph (with only primary radial lines)

In [46]:
path = mg.find_longitudinal_path(graph2, graph2.graph['start_nodes'][0])

In [398]:
graph_poly = mg.create_simple_polygon_graph(graph2, X1[:ts]) # graph2 is left bank
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_simple_polygon_graph(graph_poly, ax, 'left')
plt.axis('equal');

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 841/841 [00:09<00:00, 86.21it/s]


In [399]:
# plot individual polygon 'trajectories'
fig = plt.figure()
ax = fig.add_subplot(111)
for i in range(2, len(path), 3):
    radial_path = mg.find_radial_path_2(graph_poly, path[i])
    count = 0
    bank_type = 'left'
    cmap = plt.get_cmap("tab10")
    for node in radial_path:
        if 'poly' in graph_poly.nodes[node].keys():
            if graph_poly.nodes[node]['poly']:
                if bank_type == 'left':
                    if graph_poly.nodes[node]['direction'] == -1:
                        ax.fill(graph_poly.nodes[node]['poly'].exterior.xy[0], 
                                graph_poly.nodes[node]['poly'].exterior.xy[1],
                                facecolor = cmap(1), edgecolor='k', 
                                linewidth = 0.3, alpha = 0.5, zorder = count)
                    if graph_poly.nodes[node]['direction'] == 1:
                        ax.fill(graph_poly.nodes[node]['poly'].exterior.xy[0],
                                     graph_poly.nodes[node]['poly'].exterior.xy[1],
                                     facecolor = cmap(0), edgecolor='k', 
                                     linewidth = 0.3, alpha = 0.5, zorder = count)
        count += 1
plt.axis('equal');                    

In [525]:
polys = []
migr_rates = []
directions = []
ages = []
for node in poly_graph_1.nodes:
    polys.append(poly_graph_1.nodes[node]['poly'])
    migr_rates.append(poly_graph_1.nodes[node]['migr_rate']*poly_graph_1.nodes[node]['direction'])
    directions.append(poly_graph_1.nodes[node]['direction'])
    ages.append(poly_graph_1.nodes[node]['age'])
gdf = gpd.GeoDataFrame(polys, columns = ['geometry'])
gdf['migr_rate'] = migr_rates
gdf['direction'] = directions
gdf['age'] = ages
gdf['area'] = gdf.area
gdf.head()

,geometry,migr_rate,direction,age,area
0,"POLYGON ((280904.628 -1669530.555, 280940.756 ...",19.841002,1,0,812.806173
1,"POLYGON ((280940.756 -1669513.309, 280977.269 ...",10.012592,1,0,405.830051
2,"POLYGON ((280977.269 -1669496.886, 281014.475 ...",4.554054,1,0,181.183133
3,"POLYGON ((281014.475 -1669482.193, 281052.831 ...",2.035753,1,0,81.182172
4,"MULTIPOLYGON (((281052.831 -1669470.700, 28105...",-2.365115,-1,0,82.023968


In [513]:
gdf.crs = {'init' :'epsg:32620'}

In [514]:
gdf.to_file("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/mamore_right_bank.shp") # write shapefile

In [515]:
polys = []
migr_rates = []
directions = []
ages = []
for node in poly_graph_2.nodes:
    polys.append(poly_graph_2.nodes[node]['poly'])
    migr_rates.append(poly_graph_2.nodes[node]['migr_rate']*poly_graph_2.nodes[node]['direction'])
    directions.append(poly_graph_2.nodes[node]['direction'])
    ages.append(poly_graph_2.nodes[node]['age'])
gdf = gpd.GeoDataFrame(polys, columns = ['geometry'])
gdf['migr_rate'] = migr_rates
gdf['direction'] = directions
gdf['age'] = ages
gdf['area'] = gdf.area
gdf.crs = {'init' :'epsg:32620'}
gdf.head()

,geometry,migr_rate,direction,age,area
0,"POLYGON ((280963.202 -1669815.253, 281002.923 ...",57.883497,1,0,2315.029336
1,"POLYGON ((281002.923 -1669810.266, 281042.330 ...",59.605661,1,0,2384.686519
2,"POLYGON ((281042.330 -1669803.307, 281081.151 ...",59.174405,1,0,2361.458029
3,"POLYGON ((281081.151 -1669793.517, 281120.460 ...",58.909525,1,0,2411.817581
4,"POLYGON ((281120.460 -1669785.990, 281160.349 ...",59.542513,1,0,2500.168858


In [516]:
gdf.to_file("/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/mamore_left_bank.shp") # write shapefile

In [529]:
nx.write_gpickle(poly_graph_1, "/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/poly_graph_1.gpickle")

In [531]:
nx.write_gpickle(poly_graph_2, "/Users/zoltan/Dropbox/Channels/Fluvial/Mamore_cutoff/new_GIS_data/poly_graph_2.gpickle")

In [533]:
poly.is_valid

True

In [534]:
def load_landsat_image(img_folder, bands):
    image = {} # create an empty dictionary
    path = Path(img_folder)
    for band in bands:
        # landsat images end with *_RT_B#.TIF
        file = next(path.glob(f'*_T1_{band}.TIF')) # filename
        print(f'Opening file {file}')
        ds = rasterio.open(file) # open file using rasterio
        image.update({band: ds.read(1)}) # store band in dictionary
    return image, ds.transform, ds.bounds
    
def normalized_difference(img, b1, b2, eps=0.0001):
    band1 = np.where((img[b1]==0) & (img[b2]==0), np.nan, img[b1])
    band2 = np.where((img[b1]==0) & (img[b2]==0), np.nan, img[b2])
    return (band1 - band2) / (band1 + band2)

In [537]:
from pathlib import Path
import rasterio
folder = '/Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/'
img, transform, bounds = load_landsat_image(folder, ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])

Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B2.TIF
Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B3.TIF
Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B4.TIF
Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B5.TIF
Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B6.TIF
Opening file /Users/zoltan/Dropbox/P4GR/RTX_2023/landsat_data_2018/LC08_L1TP_232070_20180622_20200831_02_T1_B7.TIF


In [536]:
!which jupyter

/Users/zoltan/mambaforge/envs/meanderpy/bin/jupyter


In [555]:
rgb2 = np.stack([img['B7'], img['B5'], img['B3']], axis=-1)
rgb2 = rgb2/rgb2.max() * 2.5
plt.figure()
plt.imshow(rgb2, extent = [bounds[0], bounds[2], bounds[1], bounds[3]])

# cmap = plt.cm.get_cmap('Pastel1')
# # Generate random indices for colors
# num_colors = len(wbars)  # Number of colors to choose
# color_indices = np.random.randint(0, cmap.N, num_colors)
# # Get the individual colors
# colors = [cmap(i) for i in color_indices]

for i in range(len(wbars)):
#     color = colors[i]
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b, 0.5)
    for scroll in wbars[i].scrolls:
        plt.fill(scroll.polygon.exterior.xy[0], scroll.polygon.exterior.xy[1], 
                 facecolor=color, edgecolor='k', alpha=1)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [562]:
xlim = (275972.0840789568, 291897.266767988)
ylim = (-1671482.6104035194, -1647641.2654740224)
plt.xlim(xlim)
plt.ylim(ylim)

(-1671482.6104035194, -1647641.2654740224)

In [579]:
plt.figure()
plt.imshow(rgb2, extent = [bounds[0], bounds[2], bounds[1], bounds[3]])

# nodes = mg.find_longitudinal_path(graph1, 0)
# for node in nodes:
#     path, ages = mg.find_radial_path(graph2, node)
#     plt.plot(graph2.graph['x'][path], graph2.graph['y'][path], 'k')
    
for node in graph2.graph['start_nodes']:
    path, ages = mg.find_radial_path(graph2, node)
    plt.plot(graph2.graph['x'][path], graph2.graph['y'][path], 'k')

plt.xlim(xlim)
plt.ylim(ylim)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(-1671482.6104035194, -1647641.2654740224)

2023-06-29 14:47:12.498 python[60378:3041780] +[CATransaction synchronize] called within transaction


In [572]:
norm = mpl.colors.Normalize(vmin=-1, vmax=1)
m = mpl.cm.ScalarMappable(norm=norm, cmap='coolwarm_r')
plt.figure()
plt.imshow(rgb2, extent = [bounds[0], bounds[2], bounds[1], bounds[3]])
for node in tqdm(poly_graph_1.nodes):
        poly = poly_graph_1.nodes[node]['poly']
        if type(poly) == Polygon:
            plt.fill(poly.exterior.xy[0], poly.exterior.xy[1], 
                facecolor = m.to_rgba(poly_graph_1.nodes[node]['curv']*400), 
                edgecolor='k', linewidth=0.25, alpha=1)
            
# ch = mg.create_channel_polygon_2(X1[-1], Y1[-1], X2[-1], Y2[-1])
# plt.fill(ch.exterior.xy[0], ch.exterior.xy[1], 'b', alpha=0.2)

plt.xlim(xlim)
plt.ylim(ylim)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 27542/27542 [00:06<00:00, 4251.28it/s]


(-1671482.6104035194, -1647641.2654740224)

2023-06-29 14:21:28.855 python[60378:3041780] +[CATransaction synchronize] called within transaction


In [573]:
plt.figure(41)
plt.xlim(xlim)
plt.ylim(ylim)

(-1671482.6104035194, -1647641.2654740224)

2023-06-29 14:30:12.840 python[60378:3041780] +[CATransaction synchronize] called within transaction


In [580]:
cmap(0)

(0.984313725490196, 0.7058823529411765, 0.6823529411764706, 1.0)

In [584]:
cmap = plt.get_cmap("tab10")
plt.figure()
plt.imshow(rgb2, extent = [bounds[0], bounds[2], bounds[1], bounds[3]])
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], color=cmap(0))
    plt.plot(X2[i], Y2[i], color=cmap(1))
plt.xlim(xlim)
plt.ylim(ylim)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(-1671482.6104035194, -1647641.2654740224)

2023-06-29 14:56:23.781 python[60378:3041780] +[CATransaction synchronize] called within transaction
2023-06-29 14:56:42.564 python[60378:3041780] +[CATransaction synchronize] called within transaction
2023-06-29 14:56:47.859 python[60378:3041780] +[CATransaction synchronize] called within transaction
2023-06-29 14:56:48.961 python[60378:3041780] +[CATransaction synchronize] called within transaction
2023-06-29 14:56:49.231 python[60378:3041780] +[CATransaction synchronize] called within transaction
